In [115]:
import pdfplumber
import pandas as pd
from sentence_transformers import SentenceTransformer, util
import pickle
import numpy as np
import re


In [175]:
import dtale
#handy pandas visualizer, when on local Jupyter Server just run dtale.show(df)
#from Docker container visual is available at http://localhost:40000/

In [195]:
path = '../datasets/resume_samples.txt'
df = pd.DataFrame()
counter = 0
id_list = []
title_list = []
skill_list = []
#exp_list = []
with open(path, 'r',encoding="utf8", errors='backslashreplace') as f:
    for line in f.readlines():
        
        splitted = line.split(':::')
        title = splitted[1].split(';')[0]
        if 'data scientist' in title.lower():
            splitted = splitted[2].lower()
            splitted = splitted.replace('\\xa0',';;;')
            splitted = splitted.replace('*',';;;')
            splitted = splitted.replace('\\x95',';;;')
            splitted = splitted.replace('education',';;;')
            splitted = splitted.replace(',',';;;')
            splitted = splitted.split(';;;')
            
            for skill in splitted[1:]:
                if (len(skill)>1) & ('responsibilities:' not in skill.lower()):
                    id_list.append(counter)
                    
                    title_list.append(title)
                    skill_list.append(skill)
            counter+=1
df['id']=id_list
df['title']=title_list
df['skill']=skill_list

In [196]:
df

,id,title,skill
0,0,Data Scientist/Analytics Consultant,il currently a junior data scientist with a s...
1,0,Data Scientist/Analytics Consultant,data modeling
2,0,Data Scientist/Analytics Consultant,sql and data visualization. experienced in ma...
3,0,Data Scientist/Analytics Consultant,engineering
4,0,Data Scientist/Analytics Consultant,parallel programming and devops
...,...,...,...
18082,66,Data Scientist,operating systems all versions of windows
18083,66,Data Scientist,unix
18084,66,Data Scientist,linux
18085,66,Data Scientist,macintosh hd


In [54]:
def preprocess_for_bert(string):
    string = string.strip("\n.’:")
    string = string.strip("’")
    string = string.strip("\\n")
    string = string.replace("/"," ")
    return(string)

In [52]:
k21 = pickle.load(open('../Experiments/k21.sav', 'rb'))
objectives = []
model = SentenceTransformer('paraphrase-MiniLM-L6-v2')

In [197]:
skill_list = []
for s in df['skill'].to_list():
    skill_list.append(preprocess_for_bert(s))
embeddings = model.encode(skill_list)
clusters =  k21.predict(np.array(embeddings.tolist()))
df['cluster']=clusters

Batches:   0%|          | 0/566 [00:00<?, ?it/s]

In [56]:
k20_bigrams = pickle.load(open('../Experiments/k20_bigrams', 'rb'))

In [198]:
bigrams = []
for cluster in clusters:
    bigrams.append(k20_bigrams[cluster])
df['bigrams_k20']=bigrams

In [199]:
d=dtale.show(df)

In [200]:
dtale.instances()

To gain access to an instance object simply pass the value from 'ID' to dtale.get_instance(ID)

ID Name                                     URL
 1       http://584bcee9ce77:40000/dtale/main/1
         http://584bcee9ce77:40000/dtale/main/1
 2       http://584bcee9ce77:40000/dtale/main/2
         http://584bcee9ce77:40000/dtale/main/2
 3       http://584bcee9ce77:40000/dtale/main/3
         http://584bcee9ce77:40000/dtale/main/3
 4       http://584bcee9ce77:40000/dtale/main/4
         http://584bcee9ce77:40000/dtale/main/4
 5       http://584bcee9ce77:40000/dtale/main/5
         http://584bcee9ce77:40000/dtale/main/5
 6       http://584bcee9ce77:40000/dtale/main/6
         http://584bcee9ce77:40000/dtale/main/6
 7       http://584bcee9ce77:40000/dtale/main/7
         http://584bcee9ce77:40000/dtale/main/7
 8       http://584bcee9ce77:40000/dtale/main/8
         http://584bcee9ce77:40000/dtale/main/8
 9       http://584bcee9ce77:40000/dtale/main/9
         http://584bcee9ce77:40000/dtale

In [201]:
#df['cluster'].value_counts()
for c, count in df['cluster'].value_counts().iteritems():
    print(str(c)+" "+str(count)+" "+k20_bigrams[c])
    print('-----------------------')
    
for i in range(0,20):
    if i not in df['cluster'].to_list():
         print('missing '+str(i))

3 3299 software development, software engineering, best practices, data science, experience working, processing frameworks, frameworks spark, tools (e.g., standard software, ci cd,, 
-----------------------
15 2973 data visualization, large datasets, experience data, knowledge data, data visualisation, large data, big data, understanding data, data engineering, data pipelines, 
-----------------------
11 2269 develop processes, processes tools, tools monitor, model performance, performance data, data accuracy, analyze data, data analysis, ability develop, develop experimental, 
-----------------------
19 1511 programming languages, languages python,, programming skills, python data, programming experience, python, r,, data analysis, proficient python, data science, languages (python,, 
-----------------------
18 1490 working people, problem solving, lifestyle curiosity,, curiosity, learning,, learning, sharing,, sharing, working, people share, share hunger."], think creatively, solve p

In [210]:
cluster_matrix = []

for id in np.unique(df[['id']].values):
    print(id)
    cluster_counter = []
    filtered = df[df['id']==id]
    for cluster in range(0,20):
        c=0
        for index, row in filtered.iterrows():
            if row['cluster']==cluster:
                c+=1
        cluster_counter.append(c)
    cluster_matrix.append(cluster_counter)

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66


In [252]:
df_resume  = pd.DataFrame()
df_resume['id']=np.unique(df[['id']].values)
df_resume['matrix']=cluster_matrix
#df_resume = df_resume.append(pd.DataFrame(df_resume['matrix'].tolist()))
foo=pd.DataFrame(df_resume['matrix'].tolist())
df_resume = pd.concat([df_resume,foo], axis=1)
counted = df_resume.iloc[:,2:].astype(bool).sum(axis=0).sort_values(ascending=False)

In [245]:
df_resume

,id,matrix,0,1,2,3,4,5,6,7,...,10,11,12,13,14,15,16,17,18,19
0,0,"[2, 6, 2, 17, 0, 0, 1, 7, 7, 2, 0, 17, 0, 3, 1...",2,6,2,17,0,0,1,7,...,0,17,0,3,11,11,5,4,7,7
1,1,"[0, 0, 0, 7, 1, 0, 0, 5, 2, 1, 1, 2, 0, 1, 3, ...",0,0,0,7,1,0,0,5,...,1,2,0,1,3,4,0,0,3,0
2,2,"[0, 5, 0, 14, 0, 0, 3, 2, 8, 0, 0, 13, 1, 1, 8...",0,5,0,14,0,0,3,2,...,0,13,1,1,8,5,6,1,9,5
3,3,"[6, 3, 0, 14, 0, 0, 1, 2, 6, 0, 0, 11, 2, 5, 6...",6,3,0,14,0,0,1,2,...,0,11,2,5,6,10,0,1,1,3
4,4,"[1, 0, 0, 18, 0, 1, 2, 1, 13, 1, 1, 6, 1, 0, 2...",1,0,0,18,0,1,2,1,...,1,6,1,0,2,4,0,1,2,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
62,62,"[0, 19, 2, 108, 0, 0, 4, 8, 3, 0, 7, 11, 1, 1,...",0,19,2,108,0,0,4,8,...,7,11,1,1,19,29,22,8,11,7
63,63,"[0, 12, 0, 12, 0, 0, 2, 5, 2, 1, 0, 9, 0, 3, 6...",0,12,0,12,0,0,2,5,...,0,9,0,3,6,12,4,1,13,7
64,64,"[0, 6, 0, 15, 0, 0, 1, 3, 7, 0, 0, 12, 1, 0, 1...",0,6,0,15,0,0,1,3,...,0,12,1,0,10,10,5,1,6,10
65,65,"[2, 6, 2, 17, 0, 0, 1, 7, 7, 2, 0, 17, 0, 3, 1...",2,6,2,17,0,0,1,7,...,0,17,0,3,11,11,5,4,7,7


In [265]:
for cluster, count in counted.iteritems():
    print("cluster "+str(cluster)+" was present in "+str(count/len(df_resume)*100)+"% of all resumes. Most common bigrams of this cluster are "+k20_bigrams[cluster])
    print('---------------------')

cluster 18 was present in 100.0% of all resumes. Most common bigrams of this cluster are working people, problem solving, lifestyle curiosity,, curiosity, learning,, learning, sharing,, sharing, working, people share, share hunger."], think creatively, solve problems, 
---------------------
cluster 3 was present in 100.0% of all resumes. Most common bigrams of this cluster are software development, software engineering, best practices, data science, experience working, processing frameworks, frameworks spark, tools (e.g., standard software, ci cd,, 
---------------------
cluster 14 was present in 100.0% of all resumes. Most common bigrams of this cluster are international environment, work environment, relocation packages, city center, great job,, job, international, willing travel, relocation support, flexible enthusiastic, enthusiastic work, 
---------------------
cluster 19 was present in 98.50746268656717% of all resumes. Most common bigrams of this cluster are programming language